# 1. Crear el entorno de trabajo

In [ ]:
import numpy as np                                 # operaciones aritméticas, vectoriales y matriciales
import pandas as pd                                # visualización de datasets, relaciones, resultados, etc.
import matplotlib.pyplot as plt                    # visualización de gráficas y resultados
import re                                          # patrones
from google.colab import drive                     # importar archivos a Google Colab
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 2. Carga de datos

In [ ]:
FILE_NAME = "/gdrive/My Drive/Colab Notebooks/BBDD II/prac1/ejemplo-datos.csv"
df = pd.read_csv(FILE_NAME, sep=";", decimal=',', encoding= 'unicode_escape')
dataset = df.copy()                                 # trabajamos con una copia del dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 25 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   PATIENT ID                                1 non-null      float64
 1   EDAD/AGE                                  68 non-null     int64  
 2   SEXO/SEX                                  68 non-null     object 
 3   F_INGRESO/ADMISSION_D_ING/INPAT           68 non-null     object 
 4   UCI_DIAS/ICU_DAYS                         9 non-null      float64
 5   F_ALTA/DISCHARGE_DATE_ING                 66 non-null     object 
 6   MOTIVO_ALTA/DESTINY_DISCHARGE_ING         66 non-null     object 
 7   F_INGRESO/ADMISSION_DATE_URG/EMERG        63 non-null     object 
 8   HORA/TIME_ADMISION/ADMISSION_URG/EMERG    63 non-null     object 
 9   ESPECIALIDAD/DEPARTMENT_URG/EMERG         63 non-null     object 
 10  DIAG_URG/EMERG                          

# 3. Selección de variables de armonización

## 3.1 Funciones para eliminar y/o renombrar variables

In [ ]:
# eliminar variables que no aparecen en el codebook
def drop_variables(df, codebook):
  for variable in list(df.columns):
    if variable not in list(codebook.keys()):
      df = df.drop(variable, axis=1)
  return df

In [ ]:
# función para renombrar las columnas
def rename_df(df, codebook):
  for initial_name, final_name in codebook.items():
    df = df.rename(columns={initial_name: final_name})
  return df

## 3.2 Codebook del conjunto de datos

In [ ]:
# codebook del conjunto de datos
codebook = {"EDAD/AGE": "DMRAGEYR",
            "SEXO/SEX": "DMRGENDR",
            "F_INGRESO/ADMISSION_D_ING/INPAT": "DATAD",
            "UCI_DIAS/ICU_DAYS": "DATLGTI",
            "F_ALTA/DISCHARGE_DATE_ING": "DATDS",
            "MOTIVO_ALTA/DESTINY_DISCHARGE_ING": "DSXOS",
            "TEMP_PRIMERA/FIRST_URG/EMERG": "CSXBTPA",
            "FC/HR_PRIMERA/FIRST_URG/EMERG": "CSXCHRA",
            "GLU_PRIMERA/FIRST_URG/EMERG": "LBXSGLA",
            "SAT_02_PRIMERA/FIRST_URG/EMERG": "CSXOSTA",
            "TA_MAX_PRIMERA/FIRST/EMERG_URG": "CSXSYA",
            "TA_MIN_PRIMERA/FIRST_URG/EMERG": "CSXDIA",
            "FC/HR_ULTIMA/LAST_URG/EMERG": "CSXCHRHn",
            "TEMP_ULTIMA/LAST_URG/EMERG": "CSXBTPHn",
            "GLU_ULTIMA/LAST_URG/EMERG": "LBXSGLHn",
            "SAT_02_ULTIMA/LAST_URG/EMERG": "CSXOSTHn",
            "TA_MAX_ULTIMA/LAST_URGEMERG": "CSXSYHn",
            "TA_MIN_ULTIMA/LAST_URG/EMERG": "CSXDIHn"}

## 3.3 Primeros pasos

In [ ]:
# Eliminar variables duplicadas

dataset.drop_duplicates(inplace=True)
dataset.reset_index(drop=True, inplace=True)
print(dataset.shape)

(68, 25)


In [ ]:
# Eliminar variables que no hemos podido encontrar en el codebook

dataset = drop_variables(dataset, codebook)
print(dataset.shape)

(68, 18)


In [ ]:
# Renombrar variables que sí hemos podido encontrar en el codebook

dataset = rename_df(dataset, codebook)
print(dataset.shape)

(68, 18)


In [ ]:
dataset.head()

,DMRAGEYR,DMRGENDR,DATAD,DATLGTI,DATDS,DSXOS,CSXBTPA,CSXCHRA,LBXSGLA,CSXOSTA,CSXSYA,CSXDIA,CSXCHRHn,CSXBTPHn,LBXSGLHn,CSXOSTHn,CSXSYHn,CSXDIHn
0,73,FEMALE,06/03/2020,NaN,16/04/2019,Domicilio,35.5,67,0,92,103,61,67,35.500000,0,92,103,61
1,73,MALE,06/03/2020,3.0,16/04/2019,Domicilio,38.4,100,0,97,140,65,100,38.400002,0,97,0,0
2,21,MALE,06/03/2020,NaN,16/04/2019,Domicilio,39.2,113,0,97,0,0,113,39.200001,0,97,0,0
3,51,FEMALE,07/03/2020,NaN,17/04/2019,Domicilio,37.9,76,0,0,109,60,76,37.900002,0,0,109,60
4,61,MALE,07/03/2020,NaN,31/04/2019,Domicilio,36.9,105,0,93,0,0,105,36.900002,0,93,0,0


# 4. Procesar los datos

## 4.1 EDA: *Exploratory Data Analysis*

In [ ]:
# Información rápida de los datos
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 18 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   DMRAGEYR  68 non-null     int64  
 1   DMRGENDR  68 non-null     object 
 2   DATAD     68 non-null     object 
 3   DATLGTI   9 non-null      float64
 4   DATDS     66 non-null     object 
 5   DSXOS     66 non-null     object 
 6   CSXBTPA   68 non-null     float64
 7   CSXCHRA   68 non-null     int64  
 8   LBXSGLA   68 non-null     int64  
 9   CSXOSTA   68 non-null     int64  
 10  CSXSYA    68 non-null     int64  
 11  CSXDIA    68 non-null     int64  
 12  CSXCHRHn  68 non-null     int64  
 13  CSXBTPHn  68 non-null     float64
 14  LBXSGLHn  68 non-null     int64  
 15  CSXOSTHn  68 non-null     int64  
 16  CSXSYHn   68 non-null     int64  
 17  CSXDIHn   68 non-null     int64  
dtypes: float64(3), int64(11), object(4)
memory usage: 9.7+ KB


**Comentarios**. Tenemos 4 variables categóricas y 13 variables cuantitativas. Todas las variables poseen 68 valores no nulos a excepción de la variable `DATLGTI` que posee tan sólo nueve valores no nulos.

In [ ]:
dataset["DATLGTI"]

0      NaN
1      3.0
2      NaN
3      NaN
4      NaN
      ... 
63     NaN
64     NaN
65     NaN
66    16.0
67     0.0
Name: DATLGTI, Length: 68, dtype: float64

**Comentarios**. `"UCI_DIAS/ICU_DAYS": "DATLGTI"` hace referencia al número de días que un(a) paciente determinado ha estado en la UCI (Unidad de Cuidados Intensivos). La presencia de `NaN` sugiere barajear dos opciones: (1) el paciente no ha ingresado en UCI o (2) ha podido ser un error humano. Podemos partir de la suposición de que no han habido errores humanos y que, efectivamente, los `NaN` equivalen a no haber sido ingresada en la UCI. Se podrían cambiar estos valores directamente por cero.

In [ ]:
# variables categóricas
variables_cat = ['DMRGENDR', 'DSXOS', 'DATAD', 'DATDS']

# variables cuantitativas
variables_num = ['DMRAGEYR', 'DATLGTI', 'CSXBTPA', 'CSXCHRA', 'LBXSGLA', 'CSXOSTA', 'CSXSYA', 'CSXDIA',
                 'CSXCHRHn', 'CSXBTPHn', 'LBXSGLHn', 'CSXOSTHn', 'CSXSYHn', 'CSXDIHn']

### 4.2.1 Variables cuantitativas

In [ ]:
# Datos estadísticos para las variables cuantitativas
dataset.describe()

,DMRAGEYR,DATLGTI,CSXBTPA,CSXCHRA,LBXSGLA,CSXOSTA,CSXSYA,CSXDIA,CSXCHRHn,CSXBTPHn,LBXSGLHn,CSXOSTHn,CSXSYHn,CSXDIHn
count,68.000000,9.000000,68.000000,68.000000,68.0,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000,68.000000
mean,64.897059,8.111111,22.919118,55.367647,0.0,55.220588,62.000000,35.485294,71.764706,29.530882,6.352941,71.367647,82.500000,46.955882
std,16.130198,12.594090,18.184668,44.079611,0.0,45.514257,63.764726,36.919231,37.608001,15.176201,41.547586,38.903595,61.241021,35.198556
min,21.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,53.750000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,67.000000,35.875000,0.000000,71.250000,0.000000,0.000000
50%,69.000000,1.000000,36.400000,76.000000,0.0,85.500000,46.500000,22.000000,85.000000,36.799999,0.000000,92.000000,108.500000,61.000000
75%,76.250000,15.000000,36.925000,90.500000,0.0,94.000000,125.000000,69.000000,98.250000,37.599998,0.000000,96.000000,130.750000,76.250000
max,96.000000,37.000000,39.800000,118.000000,0.0,99.000000,160.000000,95.000000,124.000000,39.799999,328.000000,99.000000,164.000000,95.000000


**Comentarios**. A partir del análisis estadístico, podemos observar lo siguiente:

1. La variable `LBXSGLA`, que hace referencia a los niveles de glucosa en sangre de un paciente cuando entra a admisión, tiene por valores máximo y mínimo el cero, es decir, se trata de una columna que posee todo ceros. Podemos barajar dos opciones: (1) no se han medido los niveles de glucosa a los pacientes de COVID-19 o bien (2) ha habido algún error humano o del sistema.

  Para los valores que fueran mayores que el rango máximo, no lo ponemos como nulos porque podría tratarse de una enfermedad, como la hiperglicemia. Así que, decidimos que estos valores tomarán el valor mñaximo permitido (559).

  En cuanto a los valores que fueran menores que el rango mínimo, tampoco lo ponemos como nulos, porque podría tratarse de hipoglicemia. Así que, decidimos que estos valores tomarán el valor mínimo permitido (33). Sin embargo, si el valor es negativo, lo tratamos como nulo.

2. El valor mínimo de todas las variables, a excepción de `DMRAGEYR` es cero. Por otra parte, los valores máximos divergen, lo que indica que las variables se encuentran en escalas diferentes.

In [ ]:
# Mostrar características de la variable numérica
def numerical(df, variable):
  try:
    df[variable].plot(kind='kde', title=variable)
  except: pass
  nan = df[variable].isna().sum()
  print("Number of NaN values:", nan)

**Políticas de diseño en relación a las variables cuantitativas**. Atendiendo a la semántica de cada variable, consideramos las siguientes decisiones a la hora de armonizar los datos:

1. Para ciertas variables, los valores que se encuentren fuera del rango de valores descrito en la función `describe()` para la distribución de densidad, se considerarán como valores nulos `NaN`, de tal forma que pueden interpretarse como errores humanos o del sistema.

2. Para ciertas variables, los valores que se encuentren fuera del rango de valores descrito en la función `describe()` para la distribución de densidad, se considerarán como valores iguales a cero, de tal forma que pueden interpretarse como ausencia del paciente en una determinada condición, por ejemplo, haber sido ingresado en la UCI.

3. Para ciertas variables, los valores iguales a cero que no parecen tener significado o semántica contextuales, es decir, que parecen carecer de sentido a pesar de ser un valor que se encuentra dentro del rango de valores arrojado por la función `describe()`, se sustituirán por valores `NaN` y su interpretación será la misma que la de la política de diseño (1).

**Comentarios a las gráficas o distribuciones de densidad**. De las gráficas de las variables cuantitativas podemos apreciar lo siguiente:

1. Las variables `DMRAGEYR` y `DATLGTI` tienen distribuciones con una ligera cola, es decir, están algo desplazadas respecto a la mediana. El resto de distribuciones parecen ser bimodales pues se pueden ver que hay dos "picos".

2. Variables a las que se aplica la política de diseño n.1: `DMRAGEYR`, `CSXBTPA`, `CSXCHRA`, `LBXSGLA`, `CSXSYA` y `CSXDIA`.

3. Variables a las que se aplica la política de diseño n.2: `DATLGTI`.

4. Variables a las que se aplica la política de diseño n3.: `CSXOSTA`.



### 4.2.2 Variables categóricas o cualitativas

In [ ]:
# Información relativa a las variables categóricas
for v in variables_cat:
  print("Variable categórica: ", v)
  print(dataset[v].value_counts())
  print()

Variable categórica:  DMRGENDR
MALE      44
FEMALE    24
Name: DMRGENDR, dtype: int64

Variable categórica:  DSXOS
Domicilio               48
Traslado al Hospital     9
Fallecimiento            8
Alta Voluntaria          1
Name: DSXOS, dtype: int64

Variable categórica:  DATAD
13/03/2020    21
10/03/2020    11
09/03/2020    10
12/03/2020     9
08/03/2020     8
18/03/2020     4
06/03/2020     3
07/03/2020     2
Name: DATAD, dtype: int64

Variable categórica:  DATDS
19/04/2019    8
16/04/2019    7
21/04/2019    5
24/04/2019    5
17/04/2019    5
13/04/2019    4
12/04/2019    4
15/04/2019    3
14/04/2019    3
11/04/2019    3
22/04/2019    2
20/04/2019    2
28/04/2019    2
18/04/2019    2
25/04/2019    2
23/04/2019    2
31/04/2019    2
05/04/2019    1
26/04/2019    1
08/04/2019    1
27/04/2019    1
04/04/2019    1
Name: DATDS, dtype: int64



**Políticas de diseño en relación a las variables cualitativas**. Atendiendo a la semántica de cada variable, consideramos las siguientes decisiones a la hora de armonizar los datos:

1. Para las variables categóricas con dos posibles valores, como `DMRGENDR` (sexo), en donde no tenemos valores nulos y siguiendo la notación del codebook, sus correspondientes valores cuantitativos serán 0 y 1.

2. Para las variables categóricas con más de dos valores posibles, en donde no tenemos valores nulos y siguiendo la notación del codebook, sus correspondientes valores cuantitativos se corresponden con números enteros positivos en orden creciente, y considerando como valor inicial el cero.

3. En el caso de tener valores fuera del rango posible de valores, se considerarán como valores nulos `NaN` y tomarán el mismo significado que el la política de diseño n.1 de las variables cuantitativas.

*Nota*. Casos excepcionales como el de la variable `DATDS` (fecha de alta), en donde las fechas de alta ocurren antes de las fechas de admisión, se asume como un error humano o bien debido al sistema. Esta interpretación depende de las usuarias que gestionan y administran la base de datos, en cooperación con el personal experto del dominio de aplicación.

## 4.2 Procesar los datos

# 5. Pasar el fichero CSV a MySQL